In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.touch_actions import TouchActions
from selenium.common.exceptions import NoSuchElementException

from random import randint
from bs4 import BeautifulSoup
import pandas as pd
import time
import json

<b>Launch a webdriver instance</b>

In [59]:
driver = webdriver.Chrome()

<b>Go to LinkedIn page and log in </b>

In [60]:
## Go to LinkedIn homepage
driver.get('http://linkedin.com')
# driver.get('https://www.eventbrite.com/d/malaysia--kuala-lumpur/events/?q=&loc=Kuala%20Lumpur%2C%20Malaysia&date=')

In [61]:
## Logging in to LinkedIn
emailElem = driver.find_element_by_id('login-email')
emailElem.send_keys('@protonmail.com')

passwordElem = driver.find_element_by_id('login-password')
passwordElem.send_keys('pw')
passwordElem.submit()

<b>List of profiles to scrape</b>

In [24]:
profiles = pd.read_excel('Phase 1 CADS segmented Alumni 12.03 ZJ V1.xlsx', sheet_name='Certified', usecols=[2])

In [25]:
profiles.head()

,Name
0,How Kok Sheng
1,Tharmaraja A/L Arumugam
2,Karrthigeyan A/L Mohanadas
3,Tan Pei Pei
4,Azurawati Binti Abdul Halim


In [26]:
profiles['Name'] = profiles['Name'].dropna().apply(lambda x: x.lower().replace('/','').replace(
    'binti','bin').replace('ap','bin').replace('al','bin'))

In [27]:
new = profiles['Name'].str.split(' bin ', n = 1, expand = True)

profiles['First Name']= new[0]
profiles['Last Name']= new[1]

profiles['Full Name'] = profiles['First Name'] + " " + profiles['Last Name']

In [28]:
profiles['Full Name'].fillna(value=profiles['Name'], inplace=True)

In [29]:
profiles.reset_index(inplace=True)

In [30]:
profiles.head()

,index,Name,First Name,Last Name,Full Name
0,0,how kok sheng,how kok sheng,None,how kok sheng
1,1,tharmaraja bin arumugam,tharmaraja,arumugam,tharmaraja arumugam
2,2,karrthigeyan bin mohanadas,karrthigeyan,mohanadas,karrthigeyan mohanadas
3,3,tan pei pei,tan pei pei,None,tan pei pei
4,4,azurawati bin abdul hbinim,azurawati,abdul hbinim,azurawati abdul hbinim


<b>Fetch page HTML source

In [31]:
def fetch_html(driver):
    return driver.page_source

def parse_html(driver, html):
    return BeautifulSoup(html,'lxml')

In [32]:
html = driver.page_source
soup = BeautifulSoup(html,'lxml')
url = driver.current_url;

<b>Extract info from HTML </b>

In [33]:
def extract_basic_info(soup, url):
    
    if soup.find('h1', {'class':'pv-top-card-section__name inline t-24 t-black t-normal'}):
        name = soup.find('h1', {'class':'pv-top-card-section__name inline t-24 t-black t-normal'}).getText().strip()
    else: name = None
        
    if soup.find('h2',{'class':'pv-top-card-section__headline mt1 t-18 t-black t-normal'}):
        self_intro1 = soup.find('h2',{'class':'pv-top-card-section__headline mt1 t-18 t-black t-normal'}).getText().strip()
    else: self_intro1 = None
    
    if soup.find('h3', {'class':'pv-entity__school-name t-16 t-black t-bold'}):
        school = soup.find('h3', {'class':'pv-entity__school-name t-16 t-black t-bold'}).getText()
    else: school = None
        
    if soup.find('span', {'class':'pv-entity__comma-item'}):
        degree = soup.find('span', {'class':'pv-entity__comma-item'}).getText()
    else: degree = None

    if soup.find('span',{'class':'pv-entity__secondary-title'}):
        current_company = soup.find('span',{'class':'pv-entity__secondary-title'}).getText()
    else: current_company = None
        
    if soup.find('h3',{'class':'t-16 t-black t-bold'}):
        current_title = soup.find('h3',{'class':'t-16 t-black t-bold'}).getText()
    else: current_title = None
    
    if soup.find('p',{'class':'pv-entity__description t-14 t-black t-normal ember-view'}):
        desc = soup.find('p',{'class':'pv-entity__description t-14 t-black t-normal ember-view'}).getText()
    else: desc = None
        
    return {"url": [url],
            "name": [name],
            "school": [school],
            "degree": [degree],
            "company": [current_company],
            "title": [current_title],
            "desc": [desc]}

In [34]:
dict_profile = extract_basic_info(soup, url)
dict_profile
df = pd.DataFrame.from_dict(dict_profile)
df

,url,name,school,degree,company,title,desc
0,https://www.linkedin.com/feed/,None,None,None,None,None,None


### Search

In [35]:
def search_num(driver):
    lst_Count= driver.find_elements_by_xpath("//div[@class='search-result__wrapper']")
    return len(lst_Count)

def search(name, driver):
    searchElem = driver.find_element_by_xpath("//div[@id='nav-typeahead-wormhole']//input[1]")
    searchElem.clear()
    searchElem.send_keys(name)

    searchElem.send_keys(Keys.ENTER);
    lst_Count= driver.find_elements_by_xpath("//div[@class='search-result__wrapper']")
    
    if len(lst_Count) == 1:
        search = driver.find_element_by_css_selector('h3.actor-name-with-distance')
        search.click()
    else:
        print("Lot's of searches")


In [49]:
def new_tab(name):
    print('searching ' + name + "'s profile")
    url_link = "https://www.linkedin.com/search/results/all/?keywords=" + name + "=GLOBAL_SEARCH_HEADER"
    
    driver.execute_script("window.open('about:blank');")
    window_list = driver.window_handles
    driver.switch_to.window(window_list[-1])
    driver.get(url_link)

In [65]:
def search_new_tab(name):
    print('searching ' + name + "'s profile")
    url_link = "https://www.linkedin.com/search/results/all/?keywords=" + name + "=GLOBAL_SEARCH_HEADER"
    
    driver.execute_script("window.open('about:blank');")
    window_list = driver.window_handles
    driver.switch_to.window(window_list[-1])
    driver.get(url_link)
    
    lst_Count= driver.find_elements_by_xpath("//div[@class='search-result__wrapper']")
    if len(lst_Count) == 1:
        time.sleep(randint(0,5))
        search = driver.find_element_by_css_selector('h3.actor-name-with-distance')
        search.click()
    

In [66]:
x = 0
y = 3

i = x*y 
j = i + y
search = profiles[i:j]
for index, row in search.iterrows():
    search_new_tab(row['Full Name'])
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     print(row['index'], row['Full Name'])


searching how kok sheng's profile
searching tharmaraja arumugam's profile
searching karrthigeyan mohanadas's profile


In [ ]:
browser=webdriver.Chrome()

#first tab
browser.get('http:/reddit.com')

#second tab
browser.execute_script("window.open('about:blank', 'tab2');")
browser.switch_to.window("tab2")
browser.get('http://bing.com')

### Tabs

In [ ]:
window_before = browser.window_handles[1]
browser.switch_to.window(window_before)

In [ ]:
Window_List = browser.window_handles
Window_List

In [ ]:
browser.execute_script("window.open('');")
Window_List = browser.window_handles
browser.switch_to.window(Window_List[-2])

In [ ]:
Window_List = driver.window_handles
print(len(Window_List))

In [ ]:
window_list = driver.window_handles
for i in range(len(window_list)):
    print(i)
    driver.switch_to.window(window_list[i])
    time.sleep(4)

In [ ]:
print(dir(driver))

## Collect data

In [69]:
window_list = driver.window_handles
df = pd.DataFrame()

for i in range(len(window_list)):
    driver.switch_to.window(window_list[i])
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    url = driver.current_url

    dict_profile = extract_basic_info(soup, url)
    df2 = pd.DataFrame.from_dict(dict_profile)
    df = df.append(df2, ignore_index=True)
    
    
    time.sleep(randint(0,5))

In [68]:
df

,url,name,school,degree,company,title,desc
0,https://www.linkedin.com/feed/,None,None,None,None,None,None
1,https://www.linkedin.com/in/hue-kok-sheng-b034...,hue kok sheng,Southern University College,Barchelor's Diploma in Information Technology,IT Programmer&Software Programmer,IT Programmer,Internship for software programmer in TicTec ...
2,https://www.linkedin.com/in/tharmaraja-arumuga...,Tharmaraja Arumugam,Institute Gemilang,Advance Diploma,Paypal Malaysia Services Sdn Bhd,Compliance Executive,None
3,https://www.linkedin.com/in/karrthigeyan-mohan...,Karrthigeyan Mohanadas,Curtin University of Technology,Bachelor of Commerce,Citi,Assistant Manager - Business Fraud Analyst,- Data Analytic\n\n- Data Modeling (Incl: Dec...
4,https://www.linkedin.com/search/results/all/?k...,None,None,None,None,None,None
5,https://www.linkedin.com/search/results/all/?k...,None,None,None,None,None,None
6,https://www.linkedin.com/in/karrthigeyan-mohan...,Karrthigeyan Mohanadas,Curtin University of Technology,Bachelor of Commerce,Citi,Assistant Manager - Business Fraud Analyst,- Data Analytic\n\n- Data Modeling (Incl: Dec...


<h1>Main</h1>

In [ ]:
start_time = time.time()

random_num = randint(0,12)
time.sleep(random_num)

driver = webdriver.Chrome()
driver.get('http://linkedin.com')


# webdriver working, navigating content
name = "xun wei yee"
print('searching ' + name + "'s profile")
search(name)

currentURL = driver.getCurrentUrl();

# show_education(driver)
# show_all_skills(driver)
# show_recommendations(driver)
# show_accomplishment(driver)
# print('exposed all content')

# # fetch and parse html
# htmlsource = fetch_html(driver)
# soup = parse_html(driver, htmlsource)
# print(pid + ' html file acquired') 

# # extract data from html
# basicinfo = extract_basic_info(soup)
# dict_experience = extract_working_experience(soup)
# dict_education = extract_education(soup)
# dict_skills = extract_skills(soup)
# dict_accomplishments = extract_accomplishments(soup)
# dict_recommendations = extract_recommendations(soup)

# # put all data in a single dictionary
# profile = {}
# profile.update({'basic_info':basicinfo,
#               'working_experience':dict_experience,
#               'education':dict_education,
#               'skills':dict_skills,
#               'accomplishments':dict_accomplishments,
#               'recommendations_by_others':dict_recommendations})


# # write and export as json file
# write_to_json(profile, pid)
# print(pid + ' data collected')
   
print("--- %s seconds ---" % (time.time() - start_time)) 

In [ ]:
start_time = time.time()


for pid in to_scrape2_list[45:]:
    
    # webdriver working, navigating content
    open_profile(driver, pid)
    print('opened ' + pid + "'s profile")
    try:
        show_working_experience(driver)
    except:
        not_collected.append(pid)
        print(pid + "'s profile not collected")
        pass
        
    show_education(driver)
    show_all_skills(driver)
    show_recommendations(driver)
    show_accomplishment(driver)
    print('exposed all content')
    
    # fetch and parse html
    htmlsource = fetch_html(driver)
    soup = parse_html(driver, htmlsource)
    print(pid + ' html file acquired') 
    
    # extract data from html
    basicinfo = extract_basic_info(soup)
    dict_experience = extract_working_experience(soup)
    dict_education = extract_education(soup)
    dict_skills = extract_skills(soup)
    dict_accomplishments = extract_accomplishments(soup)
    dict_recommendations = extract_recommendations(soup)
    
    # put all data in a single dictionary
    profile = {}
    profile.update({'basic_info':basicinfo,
                  'working_experience':dict_experience,
                  'education':dict_education,
                  'skills':dict_skills,
                  'accomplishments':dict_accomplishments,
                  'recommendations_by_others':dict_recommendations})
    
    
    # write and export as json file
    write_to_json(profile, pid)
    print(pid + ' data collected')
   
print("--- %s seconds ---" % (time.time() - start_time)) 

In [ ]:
# # From the "People Also Viewed" profiles at the right, choose one randomly to click into

# def view_random_other_profile(driver):
#     random_num = randint(0,9)
#     random_profile = driver.find_elements_by_xpath("//li[@class='pv-browsemap-section__member-container mt4']")[random_num]
#     random_profile.click()

In [ ]:
# view_random_other_profile(driver)

In [ ]:
# Mimicking some human behaviors when visiting websites
driver.back() # go back to previous page

In [ ]:
def handle_one():
  return 1

def handle_two():
  return 2

def handle_three():
  return 3


dd={'one': handle_one, 
 'two': handle_two, 
 'three': handle_three}


## Testing

In [ ]:
# driver.get('https://www.linkedin.com/feed/')

searchElem = driver.find_element_by_xpath("//div[@id='nav-typeahead-wormhole']//input[1]")
searchElem.send_keys('prasanta')

searchElem.send_keys(Keys.ENTER);

In [ ]:
# search = driver.find_element_by_xpath("//section[@class='pv-profile-section education-section ember-view']")
search = driver.find_element_by_css_selector('h3.actor-name-with-distance')
search.click()

# driver.execute_script("return arguments[0].scrollIntoView();", education)
# time.sleep(2)

# try:
#     show_more_education = driver.find_element_by_xpath("//section[@class='pv-profile-section education-section ember-view']//button[@class='pv-profile-section__see-more-inline link']")
#     show_more_education.click()
# except NoSuchElementException:
#     pass

# time.sleep(5)

<b>Enter LinkedIn individual profile page</b>

In [ ]:
driver.get('https://www.linkedin.com/in/xunweiyee/')
# driver.get('http://linkedin.com/in/' + to_scrape['LinkedIn Id'].values[0])
time.sleep(5)

# scroll to bottom of page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(4)
driver.execute_script("window.scrollTo(0, 0);")

<b>Expose all content hide under "show more" button

In [ ]:
def show_working_experience(driver):
    experience = driver.find_element_by_xpath("//section[@id='experience-section']")
    driver.execute_script("return arguments[0].scrollIntoView();", experience)
    time.sleep(2)

    try:
        show_more_work_experience = driver.find_element_by_xpath("//section[@class='pv-profile-section experience-section ember-view']//button[@class='pv-profile-section__see-more-inline link']")
        show_more_work_experience.click()
    except NoSuchElementException:
        pass

    time.sleep(4)
    
def show_education(driver):
    education = driver.find_element_by_xpath("//section[@class='pv-profile-section education-section ember-view']")
    driver.execute_script("return arguments[0].scrollIntoView();", education)
    time.sleep(2)

    try:
        show_more_education = driver.find_element_by_xpath("//section[@class='pv-profile-section education-section ember-view']//button[@class='pv-profile-section__see-more-inline link']")
        show_more_education.click()
    except NoSuchElementException:
        pass

    time.sleep(5)

def show_all_skills(driver):
    try:
        skills = driver.find_element_by_xpath("//section[@class='pv-profile-section pv-featured-skills-section artdeco-container-card ember-view']")
    except NoSuchElementException:
        skills = driver.find_element_by_xpath("//section[@class='pv-profile-section pv-skill-categories-section artdeco-container-card ember-view']")

    driver.execute_script("return arguments[0].scrollIntoView();", skills)
    time.sleep(2)

    try:
        show_more_skills = driver.find_element_by_xpath("//button[@class='pv-profile-section__card-action-bar pv-skills-section__additional-skills artdeco-container-card-action-bar']")
        show_more_skills.click()
    except NoSuchElementException:
        pass

    time.sleep(5)
    
def show_recommendations(driver):## Show all recommendations
    recommendations = driver.find_element_by_xpath("//div[@class='recommendations-inlining']")
    driver.execute_script("return arguments[0].scrollIntoView();", recommendations)
    time.sleep(2)

    try:
        show_more_recommendations = driver.find_element_by_xpath("//button[@aria-controls='recommendation-list']")
        show_more_recommendations.click()
    except NoSuchElementException:
        pass

    time.sleep(4)
    
def show_accomplishment(driver):
    # scroll to bottom of page
    accomplishments = driver.find_element_by_xpath("//section[@class='pv-profile-section pv-accomplishments-section artdeco-container-card ember-view']")
    driver.execute_script("return arguments[0].scrollIntoView();", accomplishments)
    time.sleep(1)

    ## Show all accomplishments

    for expand_button in driver.find_elements_by_xpath("//section[@class='pv-profile-section pv-accomplishments-section artdeco-container-card ember-view']//button[@class='pv-accomplishments-block__expand']"):
        expand_button.click()
        time.sleep(randint(3,5))

        try:
            see_more_button = driver.find_element_by_xpath("//section[@class='pv-profile-section pv-accomplishments-section artdeco-container-card ember-view']//button[@class='pv-profile-section__see-more-inline link']")
            while see_more_button:
                see_more_button.click()
                time.sleep(7)
            else:
                expand_button.click()
                time.sleep(1)

        except:
            pass
